In [1]:
import numpy as np
import pandas as pd
import os
import math

# Load into dataframes

In [2]:
def getFilepath(fn):
    """ gets filepath for file in data folder"""
    cwd_list = os.getcwd().split('\\')
    fp = '\\'.join(cwd_list[:-1] + ['data', fn])
    return  os.path.normpath(fp)

In [3]:
def loadVixCurrent(fn='vixcurrent.csv'):
    fp = getFilepath(fn)
    df = pd.read_csv(fp, header=1)
    return df

In [4]:
def cleanVixCurrent(df):
    return df.drop(labels = ["VIX Open", "VIX High", "VIX Low"],
                  axis = 'columns')

In [5]:
def makeVixCurrent():
    return cleanVixCurrent(loadVixCurrent())

# date conversion
m/dd/YYYY -> YYYY-mm-dd

In [6]:
# Useful for making dates in twitter query format

def vixCurrent_date2twitter_format(date):
    """ "m/dd/YYYY' -> 'YYYY-mm-dd' """
    date = date.split("/")
    for i in range(2):
        if len(date[i]) == 1:
            date[i] = '0'+date[i]
    date = date[2] + '-' + date[0] + '-' + date[1]
    return date

def update_date_column(df):
    new_index = [vixCurrent_date2twitter_format(x) for x in df.loc[:,'Date']]
    df.loc[:,'Date'] = new_index
    return df

def filter_df_byDate(df, start=False, end=False):
    '''start='YYYY-MM-DD' '''
    assert start != end
    #assert format of start,end
    if start and end:
        df.query("Date >= "+start+" & Date <= "+end+" ", inplace=True)
    elif start:
        df.query("Date >= "+start, inplace=True)
    elif end:
        df.query("Date <= "+end, inplace=True)


In [7]:
#vix_df_small.head()

# make price changes column

In [8]:
def update_df_priceChange(df):
    x0 = df.loc[:len(df)-2]['VIX Close'].values
    x1 = df.loc[1:]['VIX Close'].values
    delta = np.divide(np.subtract(x1,x0), x0)
    delta = [np.nan] + list(delta)
    df.insert(2, "Daily Price Change", delta)
    df = df.drop(0)
    return df.reset_index(drop=True)

# Which to include

In [9]:
def update_df_inclusion(df, n):
    ''' n : how many to use'''
    df.sort_values(by=['Daily Price Change'], inplace=True)
    include = np.zeros(len(df))
    include[:math.floor(n/3)] = 1
    include[len(df) - math.floor(n/3):] = 1
    include[math.floor(len(df)/2 - n/6):math.floor(len(df)/2 + n/6)] = 1
    df.insert(3, "Include", include)
    return df.sort_index()

## Run and display

In [11]:
##
vix_df_all = makeVixCurrent()
vix_df_all = update_date_column(vix_df_all)
vix_df_all = update_df_priceChange(vix_df_all)
vix_df_all = update_df_inclusion(vix_df_all, 100)
filter_df_byDate(vix_df_all, start="'2008-01-01'")
vix_df_all[vix_df_all['Include'] == 1].reset_index(drop=True)

,Date,VIX Close,Daily Price Change,Include
0,2008-01-07,23.79,-0.006266,1.0
1,2008-03-18,25.79,-0.200062,1.0
2,2008-09-11,24.39,-0.005302,1.0
3,2008-09-29,46.72,0.344847,1.0
4,2008-10-13,54.99,-0.213867,1.0
5,2008-10-20,52.97,-0.246836,1.0
6,2008-10-22,69.65,0.311429,1.0
7,2009-01-21,46.42,-0.180583,1.0
8,2009-04-02,42.04,-0.005676,1.0
9,2009-10-14,22.86,-0.005655,1.0
